In [1]:
%load_ext autoreload
%autoreload 2
import env
import policy

In [2]:
e = env.Env()

In [3]:
actor = policy.HumanPolicy()
opponent = policy.NNCheckpointPolicy(checkpoint_path='./summary/nn_model_ep_500_actor.ckpt')

current_player = env.PLAYER1
while not e.game_over():
    if current_player == env.PLAYER1:
        a = actor(e.state)
        e.step(a)
    else:
        a = opponent(e.state)
        e.step(a)
    
    if not e.game_over():
        current_player = env.opponent(current_player)

if e.winner() == env.PLAYER1:
    print("You win!")
elif e.winner() == env.PLAYER2:
    print("You lose!")
else:
    print("Draw!")
print("Final board:")
env.print_state(e.state)

              
              
              
              
              
              

0 1 2 3 4 5 6
legal mask: [ True  True  True  True  True  True  True]
              
              
              
              
              
#       O     

0 1 2 3 4 5 6
legal mask: [ True  True  True  True  True  True  True]
              
              
              
              
#             
#       O O   

0 1 2 3 4 5 6
legal mask: [ True  True  True  True  True  True  True]
              
              
              
#             
#       O     
#       O O   

0 1 2 3 4 5 6
legal mask: [ True  True  True  True  True  True  True]
You win!
Final board:
              
              
O             
O             
O       #     
O       # #   



In [4]:
import network
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
policy_net = network.Actor(env.BOARD_XSIZE, env.BOARD_YSIZE).to(device)
policy_net.load_state_dict(torch.load('./summary/nn_model_ep_500_actor.ckpt'))
policy_net.eval()

Actor(
  (conv1): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=768, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=7, bias=True)
)

In [5]:
e = env.Env()

# simulate a game
e.step(env.Action(1))
e.step(env.Action(5))
e.step(env.Action(1))
e.step(env.Action(5))
e.step(env.Action(1))
e.step(env.Action(5))


# see what the network thinks of the current state
print(e.state.legal_mask())
env.print_state(e.state)
print('0 1 2 3 4 5 6 7')
print(policy_net.forward(network.state_batch_to_tensor([e.state], device))[0])

[ True  True  True  True  True  True  True]
              
              
              
  #       O   
  #       O   
  #       O   

0 1 2 3 4 5 6 7
tensor([0.1322, 0.2072, 0.1468, 0.1449, 0.1328, 0.1133, 0.1227],
       device='cuda:0', grad_fn=<SelectBackward0>)
